# Simple Script to Query DEID CDW

In [6]:
# import necessary packages
import numpy as np # numerical compute package
import pandas as pd # Data management package used for data cleaning
import pyodbc # odbc for SQL queries
import keyring
import datetime # for working with Dates and Times
import matplotlib.pyplot as plt # Markdown plotting package
%matplotlib inline
import re # Regex to do regular expression searches for data cleanup
import seaborn as sns

from tqdm import tqdm

## Part 1: SQL Queries

In [7]:
# check drivers
pyodbc.drivers()

['ODBC Driver 17 for SQL Server', 'FreeTDS']

#### Note regarding Keyring.
Keyring is a package to keep passwords so they don't need to be saved as passwords. It will need to be configured differently depending on Linux vs Mac, as well as the specific operating system. To start, you can put your free text password, but just ensure it isn't pushed to GitHub

In [8]:
# initialize connection usin
conn = pyodbc.connect(
 r'DRIVER={FreeTDS};' +
 ('SERVER={server},{port};' +
 'DATABASE={database};' +
 'UID={username};' +
 'PWD={password}').format(
 server= 'QCDIDDWDB001.ucsfmedicalcenter.org',
 port= 1433,
 database= 'CDW_NEW',
 username= 'CAMPUS\\bhuang3',
 password= keyring.get_password("deid_cdw","CAMPUS\\bhuang3"))
)

### Test SQL engine Connection

In [9]:
# Test simple connection - Total number of patients
cursor = conn.cursor()
cursor.execute("SELECT top 10 * FROM deid_uf.EncounterFact")
tb2=cursor.fetchall()
print('EncounterFactExample',tb2[0])

EncounterFactExample ('deid_uf', 'D316E19BBC6E25', 'DC0323BC1A39FE', 'DE1BF70BB68850', -1, '*Unspecified', -1, '*Unspecified', '*Unspecified', '*Unspecified', '*Unspecified', '*Unspecified', -1, 589360, 'Self-Pay', 21697004, None, 20210504, datetime.date(2021, 5, 4), 20210504, datetime.date(2021, 5, 4), -1, None, -1, None, 697256, 'Physician', '', -1, '*Unspecified', '*Unspecified', -1, '*Unspecified', '*Unspecified', 234, '*Not Applicable', 'HEM BMT PARN', 'Hematology and Oncology', 40, 'Patient Message', '', '*Not Applicable', '', '*Not Applicable', '', '*Not Applicable', '*Unspecified', 'DD4422B5F00BAA', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '*Unspecified', 'Complete', '*Unspecified', 0, 0, 0, 0, 0, None, 1)


In [ ]:
# Get number of patients per year
pt_per_yr=pd.read_sql('''SELECT count(DISTINCT PatientDurableKey) as Patients, year(EncounterFact.DateKeyValue) as VisitYear
FROM deid_uf.EncounterFact
Group By year(EncounterFact.DateKeyValue)
Order by year(EncounterFact.DateKeyValue) desc''',conn)
pt_per_yr.head()
pt_per_yr_filt=pt_per_yr[pt_per_yr.VisitYear<2022].reset_index(drop=True)

### As an example query, get VBG vs Serum Potassium from lab

In [10]:
# Test sample create table
cursor.execute('''
SELECT LabComponentKey into ##PotassiumVBG
FROM LabComponentDim
WHERE CommonName in ('POTASSIUM (BLOOD GAS)','POTASSIUMWHOLEBLOODGEM','POTASSIUM, WHOLE BLOOD VENOUS')
''')

cursor.execute('''
SELECT LabComponentKey into ##Potassium
FROM LabComponentDim
WHERE CommonName = 'POTASSIUM'
''')



In [11]:
potassium_serum_list=pd.read_sql('''SELECT * From ##Potassium
''',conn).LabComponentKey.to_list()
potassium_VBG_list=pd.read_sql('''SELECT * From ##PotassiumVBG
''',conn).LabComponentKey.to_list()

In [12]:
df_encounter_pot=pd.read_sql('''
Select distinct(EncounterKey)
from LabComponentResultFact
where LabComponentResultFact.LabComponentKey in (SELECT LabComponentKey From ##Potassium)
and LabComponentResultFact.EncounterKey in (
Select distinct(EncounterKey) from LabComponentResultFact
where LabComponentResultFact.LabComponentKey in (SELECT LabComponentKey From ##PotassiumVBG)
    )
''',conn)

In [14]:
df_encounter_pot.head(10)

,EncounterKey
0,DFCDA9D8FFEAF5
1,D80D7074F00340
2,D05E6E581A2CF2
3,D9DD546C95E412
4,D388BC3AD236E9
5,DB083111F8DCE4
6,D9C2E76EF5DBA8
7,D9AEFCC51F2D06
8,D9E5A67ACAF3BF
9,DF9B1BA6EDEED1
